# keras

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os

In [4]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels),(test_images,test_labels) = mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]
test_images = test_images.reshape(test_images.shape[0], 28, 28,1).astype('float32')
test_images = (test_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

In [23]:
# 构建模型
model = keras.Sequential([
    keras.layers.Conv2D(input_shape=(28,28,1),kernel_size=(5,5),
                        filters=5,strides=(2,2),padding="same",activation=tf.nn.relu),
    keras.layers.Conv2D(kernel_size=(5,5),
                        filters=5,strides=(2,2),padding="same",activation=tf.nn.relu),
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation=tf.nn.relu),
    keras.layers.Dense(10,activation=tf.nn.softmax)
])

In [24]:
# 设置模型的超参数
model.compile(optimizer=keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [25]:
# 训练模型
model.fit(train_images,train_labels,epochs=5)

Epoch 1/5
60000/60000 [==============================] - 20s 341us/sample - loss: 0.3943 - acc: 0.8798
Epoch 2/5
60000/60000 [==============================] - 20s 330us/sample - loss: 0.1472 - acc: 0.9564
Epoch 3/5
60000/60000 [==============================] - 19s 321us/sample - loss: 0.1122 - acc: 0.9660
Epoch 4/5
60000/60000 [==============================] - 21s 343us/sample - loss: 0.0943 - acc: 0.9710
Epoch 5/5
60000/60000 [==============================] - 21s 347us/sample - loss: 0.0837 - acc: 0.9740


In [26]:
# 测试模型
test_loss, test_acc = model.evaluate(test_images,test_labels)
print("Test accuracy:", test_acc)

10000/10000 [==============================] - 2s 172us/sample - loss: 0.0755 - acc: 0.9759
Test accuracy: 0.9759


In [21]:
model.save("./models/CNN_mnist.h5")

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 5)         130       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 5)           630       
_________________________________________________________________
flatten_1 (Flatten)          (None, 245)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                15744     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 17,154
Trainable params: 17,154
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, acc = new_model.evaluate(x_test,y_test)

In [ ]:
model.save_weights('./training_1/my_checkpoint')

In [ ]:
model.load_weights('./training_1/cp.ckpt')

In [ ]:
model.save("./training_1/mymodel.h5")

In [ ]:
new_model = keras.models.load_model('./training_1/mymodel.h5')

# 量化

In [1]:
import pickle 
from tensorflow import keras
import numpy as np

In [2]:
x_adv = pickle.load(open("adv90.pkl",'rb'),encoding='iso-8859-1')
y_origin = pickle.load(open('y_origin_90.pkl','rb'))
x_adv = np.array(x_adv)
x_adv = x_adv.reshape((-1,28,28))
y_origin = np.array(y_origin)

In [3]:
model = keras.models.load_model('./training_1/mymodel.h5')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
#初始AE
test_loss, test_acc = model.evaluate(x_adv,y_origin)
print("Test accuracy:", test_acc)

90/90 [==============================] - 0s 715us/sample - loss: 0.9481 - acc: 0.8222
Test accuracy: 0.82222223


In [5]:
# 不是AE
test_loss, test_acc = model.evaluate(x_test,y_test)
print("Test accuracy:", test_acc)

NameError: name 'x_test' is not defined

In [6]:
# float16 AE
x_adv = x_adv.astype('float16')
test_loss, test_acc = model.evaluate(x_adv,y_origin)
print("Test accuracy:", test_acc)

90/90 [==============================] - 0s 72us/sample - loss: 0.9481 - acc: 0.8222
Test accuracy: 0.82222223


In [7]:
# 把小于0.1的数置零的AE
for i in range(10):
    x_adv_copy = np.where(x_adv > 0.1*i, x_adv, 0)
    print("%dth" %i)
    test_loss, test_acc = model.evaluate(x_adv_copy,y_origin)

0th
90/90 [==============================] - 0s 89us/sample - loss: 0.9481 - acc: 0.8222
1th
90/90 [==============================] - 0s 78us/sample - loss: 0.9513 - acc: 0.8222
2th
90/90 [==============================] - 0s 89us/sample - loss: 0.9781 - acc: 0.8222
3th
90/90 [==============================] - 0s 83us/sample - loss: 1.0202 - acc: 0.8333
4th
90/90 [==============================] - 0s 78us/sample - loss: 1.0298 - acc: 0.8333
5th
90/90 [==============================] - 0s 67us/sample - loss: 1.0059 - acc: 0.8333
6th
90/90 [==============================] - 0s 55us/sample - loss: 1.0015 - acc: 0.8000
7th
90/90 [==============================] - 0s 61us/sample - loss: 1.0333 - acc: 0.8000
8th
90/90 [==============================] - 0s 67us/sample - loss: 1.0628 - acc: 0.7778
9th
90/90 [==============================] - 0s 55us/sample - loss: 0.9927 - acc: 0.7667


# 量化结论：当把像素值0.3以下的值置零时，对预测结果基本没影响。

In [28]:
%matplotlib inline
import matplotlib.pyplot as plot


In [29]:
x_adv[1].dtype

dtype('float16')

In [30]:
image = np.where(x_adv > 0.3, x_adv, 0)

In [33]:
plot.imshow(image[0])
plot.show()

ValueError: Unsupported dtype

<Figure size 432x288 with 1 Axes>

In [20]:
plot.imshow(x_adv[1])
plot.colorbar()
plot.show()

ValueError: Unsupported dtype

<Figure size 432x288 with 2 Axes>